In [1]:
data = []

from json import loads as load_json

with open("../data.json", 'r') as raw:
    data = load_json(raw.read())

In [2]:
from datasets import Dataset

ds = Dataset.from_list(data).shuffle(seed = 42) \
                            .train_test_split(test_size = 0.2)

In [3]:
from transformers import BertTokenizerFast, BertForSequenceClassification, Trainer, TrainingArguments

tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

In [4]:
ds

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 7601
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 1901
    })
})

In [5]:
def tokenize(data):
    return tokenizer(data['text'], padding = 'max_length', truncation = True)

train_ds = ds['train'].map(tokenize, batched=True)
test_ds  = ds['test'] .map(tokenize, batched=True)

Map:   0%|          | 0/7601 [00:00<?, ? examples/s]

Map:   0%|          | 0/1901 [00:00<?, ? examples/s]

In [6]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2).to('cuda')

bin c:\Users\ms2k\.conda\envs\ml\Lib\site-packages\bitsandbytes\libbitsandbytes_cuda121.dll


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'pre_classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
import os
os.environ['WANDB_DISABLED'] = 'true'

In [8]:
training_args = TrainingArguments(
    per_device_train_batch_size=8,
    output_dir='./output/bert',
    num_train_epochs=3,
)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_ds
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


TypeError: Trainer.__init__() got an unexpected keyword argument 'device'

In [ ]:
trainer.train()

  0%|          | 0/2853 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
results = trainer.evaluate(test_ds)
print(results)